In [76]:
# Proyecto Final Anpalisis Aplicado - Aimeé León

import numpy as np
import csv
import random as rd

print(np.__version__)

1.19.2


In [72]:
with open('crime_data.csv') as file:
    reader = csv.reader(file)


In [2]:
# Función de Prueba de los Algoritmos (Rosenbrock)
def Rosenbrock(x):
    # Aquí suponemos que x ya es una matriz (vector) de nx1
    return (1-x[0,0])**2 + 100*(x[1,0]-x[0,0]**2)**2

In [3]:
def gradiente(func,xk):
    # Código que calcula el gradiente de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk array y lo convertimos a matriz de nx1
    cambio = .000001
    n = len(xk)
    grad = np.matrix(np.zeros((n,1)))
    
    for i in range(1,n+1):
        aux = xk.copy()
        aux[i-1,0] = aux[i-1,0] + cambio
        grad[i-1,0] = round((func(aux) - func(xk))/cambio,2)
    
    return grad

In [4]:
def hessiana(func,xk):
    # Código que calcula la hessiana de la función f en el punto xk
        # Aquí estamos suponiendo que metemos un xk matrix de nx1
    cambio = .000001
    n = len(xk)
    hess = np.matrix(np.zeros((n,n)))
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i==j:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux2[i-1,0] = aux2[i-1,0] - cambio
                hess[i-1,j-1] = round((func(aux1)-2*func(xk)+func(aux2))/cambio**2,2)
            else:
                aux1 = xk.copy()
                aux2 = xk.copy()
                aux3 = xk.copy()
                
                aux1[i-1,0] = aux1[i-1,0] + cambio
                aux1[j-1,0] = aux1[j-1,0] + cambio
                
                aux2[j-1,0] = aux2[j-1,0] + cambio
                
                aux3[i-1,0] = aux3[i-1,0] + cambio
                
                hess[i-1,j-1] = round((func(aux1)-func(aux2)-func(aux3)+func(xk))/cambio**2,2)
                
    return hess

In [5]:
# Algoritmo 3.1: Backtracking Line Search

def BLS(func,xk):
    a = 1
    # Pues es la alpha que corresponde a la dirección de Newton
    c = 10**(-4)
    pk = -np.linalg.inv(hessiana(func,xk))*gradiente(func,xk)
    while func(xk+a*pk) > (func(xk) + c*a*np.transpose(gradiente(func,xk))*pk):
        a = .8*a
    return a

In [6]:
def posdef(func,xk):
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    if all(eigen > 0):
        r = 1 # cumple hessiana positiva definida
    else:
        r = 0 # no cumple hessiana positiva definida
        
    return r

In [7]:
def menor_eigen(func,xk):
    # Método que devuelve el menor de los eigenvalores de una matriz multiplicado por -1
    eigen = np.linalg.eig(hessiana(func,xk))[0]
    n = len(xk)
    min = 0
    for i in range(1,n+1):
        if eigen[i-1] < min:
            min = eigen[i-1]
    min = -min
    return min

In [8]:
# Método que devuelve el factor que le sumaremos a la Hessiana para volverla positiva definida

def factorCholesky(A,n):  
    b = .0001
    if np.min(np.diag(A))>0:
        fact = 0
    else:
        fact = -np.min(np.diag(A)) + b
    
    for i in range(1,100):
        eigen = np.linalg.eig(A + fact*np.identity(n))[0]
        if all(eigen > 0):
            r = 1
        else:
            r = 0

        if r==0: # es decir, si aún no es positiva definida
            fact = np.max([b,2*fact])
    
    return fact

In [52]:
class Optimizacion:
    
    def __init__(self,func,xk):
        self.func = func
        self.xk = xk
        
    
    def BFGS(self): # Broyden, Fletcher, Goldfarb, Shanno
        func = self.func
        xk = self.xk
        n = len(xk)
        e = .00001
        Hk = np.linalg.inv(hessiana(func,xk))
        while np.linalg.norm(gradiente(func,xk))>e:
            pk = -Hk*gradiente(func,xk)  #dirección
            xk1 = xk + BLS(func,xk)*pk
            sk = xk1 - xk
            yk = gradiente(func,xk1) - gradiente(func,xk)
            rok = float(1/(np.transpose(yk)*sk))
            Hk1 = (np.identity(n)-rok*sk*np.transpose(yk)) * Hk * (np.identity(n)-rok*yk*np.transpose(sk)) + rok*sk*np.transpose(sk)
            xk = xk1
            Hk = Hk1
    
        return xk
        
    
    def LSNM(self): # Búsqueda Lineal de Newton con modificación a la Hessiana
        func = self.func
        xk = self.xk
        n = len(xk)
        Bk = hessiana(func,xk)
        for k in range(1,100):
            s = factorCholesky(Bk,n)
            Bk = hessiana(func,xk) + s*np.identity(n)
            pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
            xk = xk + BLS(func,xk)*pk    
            
        return xk
    
    def AN(self): # Algoritmo de Newton
        func = self.func
        xk = self.xk
        n = len(xk)
        a = 1
        for k in range(1,100):
            Bk = hessiana(func,xk)
            pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
            xk = xk + a*pk    
            
        return xk
        
    def LS(self): # Búsqueda Lineal Base (Sin modificaciones a la Hessiana)
        func = self.func
        xk = self.xk
        n = len(xk)
        for k in range(1,100):
            Bk = hessiana(func,xk)
            pk = np.linalg.solve(Bk,-gradiente(func,xk)) 
            xk = xk + BLS(func,xk)*pk    
            
        return xk

In [53]:
# Prueba con la función Rosenbrock

xk = np.matrix([[1.5],[2.3]])
func = Rosenbrock
rosen = Optimizacion(func,xk)
rosen.AN()

matrix([[1.00112703],
        [1.00225041]])

In [93]:
# Definimos 5 cámaras aleatoriamente

cam = np.matrix([[rd.uniform(19.25,19.58),rd.uniform(-99.3,-98.97)],[rd.uniform(19.25,19.58),rd.uniform(-99.3,-98.97)],[rd.uniform(19.25,19.58),rd.uniform(-99.3,-98.97)],[rd.uniform(19.25,19.58),rd.uniform(-99.3,-98.97)],[rd.uniform(19.25,19.58),rd.uniform(-99.3,-98.97)]])

In [94]:
# Definimos la muestra aleatoria de 22 crímenes:
crim = np.matrix([[19.33845612,-99.29333614],[19.28642,-99.13235],[19.30516202,-99.08730605],[19.47661,-99.08438],[19.30033471,-99.22449462
],[19.31672032,-99.2626755],[19.496499,-99.08664023],[19.32926156,-99.21320602],[19.4495064,-99.05701426],[19.40462,-99.06965
],[19.37071,-99.16509],[19.2982669,-99.0769868],[19.48795486,-99.11981028],[19.36703,-99.07046],[19.29136681,-99.05987888],[19.32045,-99.08419
],[19.4977134,-99.09051181],[19.40737,-99.17669],[19.4469695,-99.16032595],[19.2784072,-99.17914144],[19.2805684,-99.197078],[19.37381101,-99.1699987]])

In [97]:
# Prueba con el ejemplo de las cámaras

# Función a optimizar:
def camaras(cam,crim,i,j):
    # Suponiendo i crímenes y j cámaras
    sum = 0
    for i in range(1,+1):
        for j in range(1,j+1):
            sum = sum + np.linalg.norm(cam[i,]-crim[j,])**2
            if i != j:
                sum = sum + 1/(np.linalg.norm(cam[i,]-cam[j,])**2)
    return sum
        

In [104]:
# Aplicamos el mejor estimador, que es LSNM:
xk = cam.copy()
func = camaras